# Попытка иттеративного процесса

При создании выпукло-двойственной задачи мы дважды используем значения $x_0, z_0$. Один раз они являются ограничениями правой части, другой - коэффициентами.

При этом нам бы хотелось иметь возможность менять ограничеия правой части и при этом оставлять неизменными коэффициенты. Тогда получится встраивать такие ограничения в симплекс метод.

In [1]:
from cylp.cy import CyClpSimplex
from cylp.py.modeling.CyLPModel import CyLPArray

import numpy as np
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
xa,xb = 0,1
za,zb = 0,1
grid_step = 0.1
xa_ext = xa - grid_step
xb_ext = xb + grid_step
za_ext = za - grid_step
zb_ext = zb + grid_step


x = np.arange(xa_ext, xb_ext, grid_step, dtype=np.double)
z = np.arange(za_ext, zb_ext, grid_step, dtype=np.double)
x,z = np.meshgrid(x, z)
y_cvx = x**2 + x * z + z**2
y_ccv = - x**2 - z**2

x = x.flatten()
z = z.flatten()
y_cvx = y_cvx.flatten()
y_ccv = y_ccv.flatten()

mask = x+z <= 1+grid_step

xs = x[mask]
zs = z[mask]
ys_pos = y_cvx[mask]
ys_neg = y_ccv[mask]

In [3]:
x_0, z_0 = 0.5, 0.5
x,z,y_pos,y_neg = xs,zs,ys_pos,ys_neg
dim_p = len(x)
rhs_p = np.array([x_0,z_0, 1], dtype=np.double)
dim_d = len(rhs_p) 

s_pos = CyClpSimplex()
u_pos = s_pos.addVariable('u', dim_p)
l_pos = s_pos.addVariable('l', dim_d)

s_neg = CyClpSimplex()
u_neg = s_neg.addVariable('u', dim_p)
l_neg = s_neg.addVariable('l', dim_d)


A_p_pos = np.vstack([y_pos,
                     x,
                     z,
                     np.ones(dim_p)])
A_p_neg = np.vstack([y_neg,
                     x,
                     z,
                     np.ones(dim_p)])

A_p_pos = np.matrix(A_p_pos)
A_p_neg = np.matrix(A_p_neg)

b_p = CyLPArray(np.hstack([0,rhs_p]))
b_d_pos = CyLPArray(y_pos)
b_d_neg = CyLPArray(y_neg)

A_d = np.hstack([x.reshape(-1, 1),
                 z.reshape(-1, 1),
                 np.ones(len(x)).reshape(-1, 1)])
A_d = np.matrix(A_d)


# A_d1 = np.matrix(np.vstack([-rhs_p,np.zeros((3,3))]))
t = np.array([0.1,0.1, 1], dtype=np.double)
A_d1 = np.matrix(np.vstack([-t,np.zeros((3,3))]))

s_pos += A_p_pos*u_pos + A_d1*l_pos  == b_p
s_neg += A_p_neg*u_neg + A_d1*l_neg  == b_p

for i in range(dim_p):
    s_pos += u_pos[i] >= 0
    s_neg += u_neg[i] >= 0

s_pos += A_d*l_pos <= b_d_pos
s_neg += A_d*l_neg >= b_d_neg

s_pos.objective = u_pos[0]
s_neg.objective = u_neg[0]

s_pos.primal()
s_neg.primal()

cond_pos = s_pos.primalVariableSolution['u']
yr_pos = np.dot(cond_pos, y_pos)

cond_neg = s_neg.primalVariableSolution['u']
yr_neg = np.dot(cond_neg, y_neg)
yr_pos + yr_neg, s_pos.getStatusString(), s_neg.getStatusString()

(1.6653345369377348e-16, 'primal infeasible', 'primal infeasible')

In [4]:
x_0, z_0 = 0.3, 0.5
x,z,y_pos,y_neg = xs,zs,ys_pos,ys_neg
dim_p = len(x)
rhs_p = np.array([x_0,z_0, 1], dtype=np.double)
dim_d = len(rhs_p) 

s_pos = CyClpSimplex()
u_pos = s_pos.addVariable('u', dim_p)
l_pos = s_pos.addVariable('l', dim_d)

s_neg = CyClpSimplex()
u_neg = s_neg.addVariable('u', dim_p)
l_neg = s_neg.addVariable('l', dim_d)


A_p_pos = np.vstack([y_pos,
                     x,
                     z,
                     np.ones(dim_p)])
A_p_neg = np.vstack([y_neg,
                     x,
                     z,
                     np.ones(dim_p)])

A_p_pos = np.matrix(A_p_pos)
A_p_neg = np.matrix(A_p_neg)

b_p = CyLPArray(np.hstack([0,rhs_p]))
b_d_pos = CyLPArray(y_pos)
b_d_neg = CyLPArray(y_neg)

A_d = np.hstack([x.reshape(-1, 1),
                 z.reshape(-1, 1),
                 np.ones(len(x)).reshape(-1, 1)])
A_d = np.matrix(A_d)


# A_d1 = np.matrix(np.vstack([-rhs_p,np.zeros((3,3))]))
t = np.array([1,1, 1], dtype=np.double)
A_d1 = np.matrix(np.vstack([-t,np.zeros((3,3))]))

s_pos += A_p_pos*u_pos + A_d1*l_pos  == b_p
s_neg += A_p_neg*u_neg + A_d1*l_neg  == b_p

for i in range(dim_p):
    s_pos += u_pos[i] >= 0
    s_neg += u_neg[i] >= 0

s_pos += A_d*l_pos <= b_d_pos
s_neg += A_d*l_neg >= b_d_neg

s_pos.objective = u_pos[0]
s_neg.objective = u_neg[0]

s_pos.primal()
s_neg.primal()

cond_pos = s_pos.primalVariableSolution['u']
yr_pos = np.dot(cond_pos, y_pos)

cond_neg = s_neg.primalVariableSolution['u']
yr_neg = np.dot(cond_neg, y_neg)
yr_pos + yr_neg, x_0*z_0, s_pos.getStatusString(), s_neg.getStatusString()

(0.20400000000000007, 0.15, 'optimal', 'optimal')

значения, которые вводятся в коэффициенты для $l$ должны быть больше чем значения, которые вносятся в коэффициенты правой части.

Если быть точнее, то судя по всему, сумма значений... хотя и это не точно.

Точность при этом хромает, но с этим нужно разбираться.

In [5]:
def f(x,z,y_pos,y_neg,x_0,z_0,x_b=1,z_b=1):
    dim_p = len(x)
    rhs_p = np.array([x_0,z_0, 1], dtype=np.double)
    dim_d = len(rhs_p) 

    s_pos = CyClpSimplex()
    u_pos = s_pos.addVariable('u', dim_p)
    l_pos = s_pos.addVariable('l', dim_d)

    s_neg = CyClpSimplex()
    u_neg = s_neg.addVariable('u', dim_p)
    l_neg = s_neg.addVariable('l', dim_d)


    A_p_pos = np.vstack([y_pos,
                         x,
                         z,
                         np.ones(dim_p)])
    A_p_neg = np.vstack([y_neg,
                         x,
                         z,
                         np.ones(dim_p)])

    A_p_pos = np.matrix(A_p_pos)
    A_p_neg = np.matrix(A_p_neg)

    b_p = CyLPArray(np.hstack([0,rhs_p]))
    b_d_pos = CyLPArray(y_pos)
    b_d_neg = CyLPArray(y_neg)

    A_d = np.hstack([x.reshape(-1, 1),
                     z.reshape(-1, 1),
                     np.ones(len(x)).reshape(-1, 1)])
    A_d = np.matrix(A_d)


    # A_d1 = np.matrix(np.vstack([-rhs_p,np.zeros((3,3))]))
    t = np.array([x_b,z_b, 1], dtype=np.double)
    A_d1 = np.matrix(np.vstack([-t,np.zeros((3,3))]))

    s_pos += A_p_pos*u_pos + A_d1*l_pos  == b_p
    s_neg += A_p_neg*u_neg + A_d1*l_neg  == b_p

    for i in range(dim_p):
        s_pos += u_pos[i] >= 0
        s_neg += u_neg[i] >= 0

    s_pos += A_d*l_pos <= b_d_pos
    s_neg += A_d*l_neg >= b_d_neg

    s_pos.objective = u_pos[0]
    s_neg.objective = u_neg[0]

    s_pos.primal()
    s_neg.primal()

    cond_pos = s_pos.primalVariableSolution['u']
    yr_pos = np.dot(cond_pos, y_pos)

    cond_neg = s_neg.primalVariableSolution['u']
    yr_neg = np.dot(cond_neg, y_neg)
    return yr_pos + yr_neg, x_0*z_0, s_pos.getStatusString(), s_neg.getStatusString(), s_pos.primalVariableSolution['u'], s_neg.primalVariableSolution['u'], s_pos.primalVariableSolution['l'], s_neg.primalVariableSolution['l']

In [6]:
r = f(xs,zs,ys_pos,ys_neg,0.1,0.1)
r

(0.020000000000000018,
 0.010000000000000002,
 'optimal',
 'optimal',
 array([0.        , 0.33333333, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.11111111, 0.        ,
        0.        , 0.55555556, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.

Проверим значения на некотором множестве точек и выясним отклонения:

In [7]:
for x in np.arange(0.1,1,0.1):
    for y in np.arange(0.1,1,0.1):
        if x+y <= 1:
            r = f(xs,zs,ys_pos,ys_neg,x,y)
            arr = [*r[-2],*r[-1]]
            condpos = r[-4]
            condneg = r[-3]
            x0p = np.dot(condpos,xs)
            z0n = np.dot(condneg,zs)
            print("{:2.1f} {:2.1f} :fx= {:+.4f}, {:+.4f}, \u0394fx: {:.4f}; {:2.1f} {:2.1f}".
                  format(x,y,r[0],x*y,abs(r[0]-x*y),x0p,z0n))

0.1 0.1 :fx= +0.0200, +0.0100, Δfx: 0.0100; 0.1 0.1
0.1 0.2 :fx= +0.0200, +0.0200, Δfx: 0.0000; 0.1 0.2
0.1 0.3 :fx= +0.0067, +0.0300, Δfx: 0.0233; 0.1 0.3
0.1 0.4 :fx= +0.0067, +0.0400, Δfx: 0.0333; 0.1 0.4
0.1 0.5 :fx= +0.0167, +0.0500, Δfx: 0.0333; 0.1 0.5
0.1 0.6 :fx= +0.0400, +0.0600, Δfx: 0.0200; 0.1 0.6
0.1 0.7 :fx= +0.0767, +0.0700, Δfx: 0.0067; 0.1 0.7
0.1 0.8 :fx= +0.0800, +0.0800, Δfx: 0.0000; 0.1 0.8
0.1 0.9 :fx= +0.0733, +0.0900, Δfx: 0.0167; 0.1 0.9
0.2 0.1 :fx= +0.0200, +0.0200, Δfx: 0.0000; 0.2 0.1
0.2 0.2 :fx= +0.0400, +0.0400, Δfx: 0.0000; 0.2 0.2
0.2 0.3 :fx= +0.0569, +0.0600, Δfx: 0.0031; 0.2 0.3
0.2 0.4 :fx= +0.0739, +0.0800, Δfx: 0.0061; 0.2 0.4
0.2 0.5 :fx= +0.0721, +0.1000, Δfx: 0.0279; 0.2 0.5
0.2 0.6 :fx= +0.1513, +0.1200, Δfx: 0.0313; 0.2 0.6
0.2 0.7 :fx= +0.1247, +0.1400, Δfx: 0.0153; 0.2 0.7
0.2 0.8 :fx= +0.1115, +0.1600, Δfx: 0.0485; 0.2 0.8
0.3 0.1 :fx= +0.0067, +0.0300, Δfx: 0.0233; 0.3 0.1
0.3 0.2 :fx= +0.0569, +0.0600, Δfx: 0.0031; 0.3 0.2
0.3 0.3 :fx=

In [8]:
for x in np.arange(0.1,1,0.1):
    for y in np.arange(0.1,1,0.1):
        if x+y <= 1:
            r = f(xs,zs,ys_pos,ys_neg,x,y,x_b=x,z_b=y)
            arr = [*r[-2],*r[-1]]
            condpos = r[-4]
            condneg = r[-3]
            x0p = np.dot(condpos,xs)
            z0n = np.dot(condneg,zs)
            print("{:2.1f} {:2.1f} :fx= {:+.4f}, {:+.4f}, \u0394fx: {:.4f}; {:2.1f} {:2.1f}".
                  format(x,y,r[0],x*y,abs(r[0]-x*y),x0p,z0n))

0.1 0.1 :fx= +0.0100, +0.0100, Δfx: 0.0000; 0.1 0.1
0.1 0.2 :fx= +0.0200, +0.0200, Δfx: 0.0000; 0.1 0.2
0.1 0.3 :fx= +0.0300, +0.0300, Δfx: 0.0000; 0.1 0.3
0.1 0.4 :fx= +0.0400, +0.0400, Δfx: 0.0000; 0.1 0.4
0.1 0.5 :fx= +0.0500, +0.0500, Δfx: 0.0000; 0.1 0.5
0.1 0.6 :fx= +0.0600, +0.0600, Δfx: 0.0000; 0.1 0.6
0.1 0.7 :fx= +0.0700, +0.0700, Δfx: 0.0000; 0.1 0.7
0.1 0.8 :fx= +0.0800, +0.0800, Δfx: 0.0000; 0.1 0.8
0.1 0.9 :fx= +0.0900, +0.0900, Δfx: 0.0000; 0.1 0.9
0.2 0.1 :fx= +0.0200, +0.0200, Δfx: 0.0000; 0.2 0.1
0.2 0.2 :fx= +0.0400, +0.0400, Δfx: 0.0000; 0.2 0.2
0.2 0.3 :fx= +0.0600, +0.0600, Δfx: 0.0000; 0.2 0.3
0.2 0.4 :fx= +0.0800, +0.0800, Δfx: 0.0000; 0.2 0.4
0.2 0.5 :fx= +0.1000, +0.1000, Δfx: 0.0000; 0.2 0.5
0.2 0.6 :fx= +0.1200, +0.1200, Δfx: 0.0000; 0.2 0.6
0.2 0.7 :fx= +0.1400, +0.1400, Δfx: 0.0000; 0.2 0.7
0.2 0.8 :fx= +0.1600, +0.1600, Δfx: 0.0000; 0.2 0.8
0.3 0.1 :fx= +0.0300, +0.0300, Δfx: 0.0000; 0.3 0.1
0.3 0.2 :fx= +0.0600, +0.0600, Δfx: 0.0000; 0.3 0.2
0.3 0.3 :fx=